## Notebook to generate Master Mapping excell for Size (subsize resolved) vs. Category &   Channel & Region 
Jul 2020 by Rafal Rakowski

In [4]:
# Create environment
import pandas as pd
from pandas import Series,DataFrame
from googletrans import Translator
import numpy as np
import re
import sys

In [5]:
# Check path
%pwd

'/Users/Raf/ecommerce_practical/Cross_Mapping_Project'

In [6]:
# Read in with API the eBay UK_Item Specifics_January_2020.xlsx into Python Pandas
master_mapper = pd.ExcelFile('eBay UK_Item Specifics_January_2020.xlsx')
data_frame = master_mapper.parse('eBay UK')

In [7]:
# Show Python imported data frame of the eBay UK_Item Specifics_January_2020.xlsx table
data_frame

,Category ID,Category Name,Item Specific Name,Required?,Allows Variations?,eBay Values Only?,Option List
0,96762,Antiques > Antique Clocks > Bracket Clocks > P...,Clock Type,No,Yes,No,"Alarm Clock, Backward Clock, Bracket Clock, Ch..."
1,96762,Antiques > Antique Clocks > Bracket Clocks > P...,Age,No,No,No,"Antique, Post-1900, Antique, Pre-1900, Antique..."
2,96762,Antiques > Antique Clocks > Bracket Clocks > P...,Type,No,No,No,Bracket Clock
3,96762,Antiques > Antique Clocks > Bracket Clocks > P...,Style/ Origin/ Theme,No,No,No,"American, Art Deco, Chinese, Edwardian, French..."
4,96762,Antiques > Antique Clocks > Bracket Clocks > P...,Material,No,No,No,"Bakelite, Brass, Bronze, Cane/ Wicker, Ceramic..."
...,...,...,...,...,...,...,...
191562,90951,Wholesale & Job Lots > Tickets & Travel,Unit Type,No,No,YES,"100g, 100ml, 10g, 10ml, kg, L, m, m², m³, Unit"
191563,90951,Wholesale & Job Lots > Tickets & Travel,Brand,No,No,No,NaN
191564,26424,Wholesale & Job Lots > Toys & Games,Unit Quantity,No,No,No,NaN
191565,26424,Wholesale & Job Lots > Toys & Games,Unit Type,No,No,YES,"100g, 100ml, 10g, 10ml, kg, L, m, m², m³, Unit"


In [8]:
# Create an empty output dataframe (to populate with processed data and to export to excel the revised here Master
# Mapping) with the key size-mapper columns
df_output = pd.DataFrame(columns=['Category ID', 'Category Name', 'Item Specific Name', 'Option List']) 

In [9]:
# Cleanse data, remove empty records
df_output['Category ID'] = data_frame['Category ID']
df_output['Category Name'] = data_frame['Category Name']
df_output['Item Specific Name'] = data_frame['Item Specific Name']
df_output['Option List'] = data_frame['Option List']
for col in df_output.columns:
    bool_series = pd.isnull(df_output[col])  
    df_output[col][bool_series] = 'empty'

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
# Introduce some key names for products associated to a size feature
product_name = 'Clothes Clothing Shoes' 
product_name_list = product_name.split(' ')
product_name_list

['Clothes', 'Clothing', 'Shoes']

In [11]:
# Match either word in new product name to records in Category Name column 
# Pick indexes for found hits and insert corresponding to them rows to the created output data frame

bla = [False] * len(df_output['Category Name'])
result = np.array(bla)

for word in product_name_list:
    print(word)
    matched = df_output['Category Name'].str.contains(r'\b{}\b'.format(word)) 
    matched = np.array(matched)
    y=any(matched.tolist())
    #print(y)
    if  y:
        result = matched | result
hits = [i for i, x in enumerate(result) if x]
df_output_temp = df_output[result]

Clothes
Clothing
Shoes


In [12]:
# See how many lines there are in the output frame (it is ~10k rows from original ~ 200k)
df_output_temp['Category Name'].size

10901

In [13]:
# Match either phrase containing 'Size' word in 'Item Specific Name' column and insert matched rows to output frame
# Py Pandas will align and remove previously insterted data rows automatically!

size_list = ['Size'] 
bla = [False] * len(df_output_temp['Item Specific Name'])
result = np.array(bla)

for word in size_list:
    print(word)
    matched = df_output_temp['Item Specific Name'].str.contains(r'\b{}\b'.format(word)) 
    matched = np.array(matched)
    y=any(matched.tolist())
    #print(y)
    if  y:
        result = matched | result
hits = [i for i, x in enumerate(result) if x]
df_output_sizes = df_output_temp[result]

Size


In [14]:
# Now there is even less data (left 780 rows)
df_output_sizes['Category Name'].size

780

In [69]:
# Show output frame
df_output_sizes

,Category ID,Category Name,Item Specific Name,Option List
18285,109674,"Business, Office & Industrial > Facility Maint...",Shoe Size,"UK 4.5 (Men's), UK 5 (Men's), UK 5.5 (Men's), ..."
44871,260022,"Clothes, Shoes & Accessories > Baby > Baby & T...",Size,"0-3 Months, 12-18 Months, 12 Months, 18-24 Mon..."
44888,260020,"Clothes, Shoes & Accessories > Baby > Baby & T...",Size,"0-3 Months, 12-18 Months, 12 Months, 18-24 Mon..."
44898,260035,"Clothes, Shoes & Accessories > Baby > Baby & T...",Size,"0-3 Months, 12-18 Months, 12 Months, 18-24 Mon..."
44909,139762,"Clothes, Shoes & Accessories > Baby > Baby & T...",Size,"Newborn, 0-3 Months, 3-6 Months, 6-9 Months, 9..."
...,...,...,...,...
178202,177106,"Vehicle Parts & Accessories > Clothing, Helmet...",Size,"34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56..."
178220,177108,"Vehicle Parts & Accessories > Clothing, Helmet...",Size,"XXS, XS, S, M, L, XL, XXL, XXXL, One Size, Child"
178238,177109,"Vehicle Parts & Accessories > Clothing, Helmet...",Size,"34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56..."
178258,177110,"Vehicle Parts & Accessories > Clothing, Helmet...",Size,"34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56..."


In [16]:
# Repeat all done above for eBay DE_Items (above was for eBay UK_Item)

# Read in with API the eBay DE_Item Specifics_January_2020.xlsx into Python Pandas
master_mapper_DE = pd.ExcelFile('eBay DE_Item Specifics_January 2020.xlsx')
data_frame_DE = master_mapper_DE.parse('eBay DE')

#Create an empty output dataframe with the key size-mapper columns
df_output_DE = pd.DataFrame(columns=['Category ID', 'Category Name', 'Item Specific Name', 'Option List'])
df_output_DE['Category ID'] = data_frame_DE['Category ID']
df_output_DE['Category Name'] = data_frame_DE['Category Name']
df_output_DE['Item Specific Name'] = data_frame_DE['Item Specific Name']
df_output_DE['Option List'] = data_frame_DE['Option List']
for col in df_output_DE.columns:
    bool_series = pd.isnull(df_output_DE[col])  
    df_output_DE[col][bool_series] = 'empty'
    
product_name_DE = 'Kleidung Bekleidung Kleider Schuhe' 
product_name_list_DE = product_name_DE.split(' ')

# Match either word in new product name to records in Category Name column 
# Pick indexes for found hits and insert corresponding to them rows to the created output data frame
bla = [False] * len(df_output_DE['Category Name'])
result = np.array(bla)

for word in product_name_list_DE:
    print(word)
    matched = df_output_DE['Category Name'].str.contains('{}'.format(word)) 
    matched = np.array(matched)
    y=any(matched.tolist())
    #print(y)
    if  y:
        result = matched | result
hits_DE = [i for i, x in enumerate(result) if x]

df_output_temp_DE = df_output_DE[result]

translator = Translator()
translations = translator.translate(['Size'], dest='de')
for translation in translations:
    term = translation.text
size_list_DE = [term] 

# Match either phrase containing 'Size' word in 'Item Specific Name' column and insert matched rows to output frame
# Py Pandas will align and remove previously insterted data rows automatically!
bla = [False] * len(df_output_temp_DE['Item Specific Name'])
result = np.array(bla)

for word in size_list_DE:
    print(word)
    matched = df_output_temp_DE['Item Specific Name'].str.contains('{}'.format(word)) 
    matched = np.array(matched)
    y=any(matched.tolist())
    #print(y)
    if  y:
        result = matched | result
hits = [i for i, x in enumerate(result) if x]
df_output_sizes_DE = df_output_temp_DE[result]

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


Kleidung
Bekleidung
Kleider
Schuhe
Größe


In [17]:
# Show the output frame for df_output_sizes_DE
df_output_sizes_DE

,Category ID,Category Name,Item Specific Name,Option List
15267,42508,Auto & Motorrad: Teile > Automobilia > Bekleid...,Größe,"XS, S, M, L, XL, XXL"
15397,73665,Auto & Motorrad: Teile > Auto-Motorsport > Bek...,Größe,"XS, S, M, L, XL, XXL, XXXL"
15413,73666,Auto & Motorrad: Teile > Auto-Motorsport > Bek...,Größe,"XS, S, M, L, XL, XXL, XXXL"
15429,73668,Auto & Motorrad: Teile > Auto-Motorsport > Bek...,Größe,"XS, S, M, L, XL, XXL, XXXL"
15445,73667,Auto & Motorrad: Teile > Auto-Motorsport > Bek...,Größe,"XS, S, M, L, XL, XXL, XXXL"
...,...,...,...,...
178065,159167,Sport > Weitere Sportarten > Turnen & Gymnasti...,Größe,"XS, S, M, L, XL, XXL"
178198,4939,Sport > Weitere Wintersportarten > Eislaufen >...,Größe,"XXS, XS, S, M, ML, L, XL, 2XL, 3XL, 4XL, 32 - ..."
178309,77617,Sport > Weiterer Wassersport > Kitesurfen > Be...,Größe,"XXS, XS, S, M, ML, L, XL, 2XL, 3XL, 4XL, 32 - ..."
178419,77624,Sport > Weiterer Wassersport > Wakeboarden > B...,Größe,"XXS, XS, S, M, ML, L, XL, 2XL, 3XL, 4XL, 32 - ..."


In [18]:
# Combine (merge) both tables 'eBay UK_Item' & 'eBay DE_Item' towards creating the Master MApping for Clothing Sizes
combined_mapper = pd.merge(df_output_sizes, df_output_sizes_DE, on = 'Category ID', how = 'outer', suffixes =('_UK', '_DE'))

In [28]:
# Show the combine table
combined_mapper

,Category ID,Category Name_UK,Item Specific Name_UK,Option List_UK,Category Name_DE,Item Specific Name_DE,Option List_DE
0,109674,"Business, Office & Industrial > Facility Maint...",Shoe Size,"UK 4.5 (Men's), UK 5 (Men's), UK 5.5 (Men's), ...",NaN,NaN,NaN
1,260022,"Clothes, Shoes & Accessories > Baby > Baby & T...",Size,"0-3 Months, 12-18 Months, 12 Months, 18-24 Mon...",Kleidung & Accessoires > Babys > Mode für Baby...,Größe,"80, 86, Bis Größe 50, Einheitsgröße, Größe 56,..."
2,260020,"Clothes, Shoes & Accessories > Baby > Baby & T...",Size,"0-3 Months, 12-18 Months, 12 Months, 18-24 Mon...",Kleidung & Accessoires > Babys > Mode für Baby...,Größe,"80, 86, Bis Größe 50, Einheitsgröße, Größe 56,..."
3,260035,"Clothes, Shoes & Accessories > Baby > Baby & T...",Size,"0-3 Months, 12-18 Months, 12 Months, 18-24 Mon...",NaN,NaN,NaN
4,139762,"Clothes, Shoes & Accessories > Baby > Baby & T...",Size,"Newborn, 0-3 Months, 3-6 Months, 6-9 Months, 9...",Kleidung & Accessoires > Babys > Mode für Baby...,Größe,"Bis Größe 50, Größe 56, Größe 62, Größe 68, Gr..."
...,...,...,...,...,...,...,...
988,159167,NaN,NaN,NaN,Sport > Weitere Sportarten > Turnen & Gymnasti...,Größe,"XS, S, M, L, XL, XXL"
989,4939,NaN,NaN,NaN,Sport > Weitere Wintersportarten > Eislaufen >...,Größe,"XXS, XS, S, M, ML, L, XL, 2XL, 3XL, 4XL, 32 - ..."
990,77617,NaN,NaN,NaN,Sport > Weiterer Wassersport > Kitesurfen > Be...,Größe,"XXS, XS, S, M, ML, L, XL, 2XL, 3XL, 4XL, 32 - ..."
991,77624,NaN,NaN,NaN,Sport > Weiterer Wassersport > Wakeboarden > B...,Größe,"XXS, XS, S, M, ML, L, XL, 2XL, 3XL, 4XL, 32 - ..."


In [19]:
# Save the combined table to an excel file (it could be saved as a sub_spreadsheet of existing Master Mapping excel)
combined_mapper.to_excel('Item Specific Size Mapper.xlsx', header = True)

In [15]:
# Translate test to be implemented later on for non_English regions
translator = Translator()
translations = translator.translate(['Size'], dest='de')
for translation in translations:
    word = translation.text

In [16]:
word

'Größe'